In [1]:
print('get qualified txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import flipside_utils as f
import clickhouse_utils as ch
import csv_utils as cu
import google_bq_utils as bqu
import opstack_metadata_utils as ops
sys.path.pop()

import time

import numpy as np
import pandas as pd
import os
import clickhouse_connect as cc

get qualified txs


In [2]:
# d = ops.get_op_stack_metadata_by_data_source('flipside')
chain_configs = ops.generate_op_stack_chain_config_query_list()

# Get All Chain IDs in our metadata list & DB
# Commented out to read from Flip & CH responses
# chain_ids_string = ops.gen_chain_ids_list_for_param(chain_configs['mainnet_chain_id'])

# display(chain_configs)

In [3]:
ch_client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

query_name = 'daily_evms_qualified_txs_counts'

In [4]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs','source'
        ]

In [5]:
trailing_days = 180

flipside_configs = chain_configs[chain_configs['source'] == 'flipside']
clickhouse_configs = chain_configs[chain_configs['source'] == 'oplabs']

In [6]:
print('     flipside runs')
flip_dfs = []

with open(os.path.join("inputs/sql/flipside_bychain.sql"), "r") as file:
    og_query = file.read()

for index, chain in flipside_configs.iterrows():
    print('     flipside: ' + chain['blockchain'])
    query = og_query
    # Pass in Params to the query
    query = query.replace("@blockchain@", chain['blockchain'])
    query = query.replace("@chain_id@", str(chain['mainnet_chain_id']))
    query = query.replace("@name@", chain['display_name'])
    query = query.replace("@layer@", chain['chain_layer'])
    query = query.replace("@trailing_days@", str(trailing_days))
    
    try:
        df = f.query_to_df(query)
        flip_dfs.append(df)
    except Exception as e:  # Use FlipsideError if available instead of Exception
        print(f"Error querying Flipside for {chain['blockchain']}: {str(e)}")
        print("Skipping this chain due to API credit limitation or other issues.")
        continue

if flip_dfs:
    flip = pd.concat(flip_dfs)
    flip['source'] = 'flipside'
    flip['dt'] = pd.to_datetime(flip['dt']).dt.tz_localize(None)
    flip = flip[col_list]
else:
    print("No data was retrieved from Flipside. The resulting DataFrame will be empty.")
    flip = pd.DataFrame(columns=col_list)

     flipside runs
     flipside: blast


In [7]:
# Run Clickhouse
print('     clickhouse runs')
ch_dfs = []
with open(os.path.join("inputs/sql/goldsky_bychain.sql"), "r") as file:
                        og_query = file.read()

for index, chain in clickhouse_configs.iterrows():
        print(     'clickhouse: ' + chain['blockchain'])
        query = og_query
        #Pass in Params to the query
        query = query.replace("@blockchain@", chain['blockchain'])
        if chain['blockchain'] == 'bob':
                query = query.replace("chain_id, --db chain_id", str(chain['mainnet_chain_id']) + " as chain_id,")
                # query = query.replace("@chain_id@", str(chain['mainnet_chain_id']))
        query = query.replace("@name@", chain['display_name'])
        query = query.replace("@layer@", chain['chain_layer'])
        query = query.replace("@trailing_days@", str(trailing_days))
        try:
                df = ch_client.query_df(query)
                ch_dfs.append(df)
        except:
                print('unable to process ' + chain['blockchain'])

ch = pd.concat(ch_dfs)
ch['source'] = 'goldsky'
ch['dt'] = pd.to_datetime(ch['dt']).dt.tz_localize(None)
ch = ch[col_list]

     clickhouse runs
clickhouse: op
clickhouse: zora
clickhouse: base
clickhouse: mode
clickhouse: lisk
clickhouse: metal
clickhouse: mint
clickhouse: fraxtal
clickhouse: redstone
clickhouse: cyber
clickhouse: swan


In [8]:
# Get Chains we already have data for & don't need to run in Dune
dataframes = [ch, flip]

chain_ids_string = ops.get_unique_chain_ids_from_dfs(dataframes)

690,185,1135,7560,254,8453,1750,252,7777777,10,34443,81457


In [9]:
# Run Dune
print('     dune runs')
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
chain_ids_param = d.generate_query_parameter(input=chain_ids_string,field_name='list_chain_ids',dtype='text')

dune_df = d.get_dune_data(query_id = 3740822, #https://dune.com/queries/3740822
    name = "dune_evms_qualified_txs",
    path = "outputs",
    performance="large",
    params = [days_param,chain_ids_param],
    num_hours_to_rerun=0 #always rerun due to param
)
dune_df['source'] = 'dune'
dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
dune_df = dune_df[col_list]

2024-08-16 12:19:57.342 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3740822?trailing_days=180&list_chain_ids=690%2C185%2C1135%2C7560%2C254%2C8453%2C1750%2C252%2C7777777%2C10%2C34443%2C81457


     dune runs


2024-08-16 12:19:58,411 INFO dune_client.api.base executing 3740822 on large cluster
2024-08-16 12:19:58,814 INFO dune_client.api.base waiting for query execution 01J5DZT5DN88FGSMNM892W0YRQ to complete: ExecutionState.PENDING (queue position: 1)
2024-08-16 12:19:59,955 INFO dune_client.api.base waiting for query execution 01J5DZT5DN88FGSMNM892W0YRQ to complete: ExecutionState.PENDING (queue position: 1)
2024-08-16 12:20:01,101 INFO dune_client.api.base waiting for query execution 01J5DZT5DN88FGSMNM892W0YRQ to complete: ExecutionState.PENDING (queue position: 1)
2024-08-16 12:20:02,296 INFO dune_client.api.base waiting for query execution 01J5DZT5DN88FGSMNM892W0YRQ to complete: ExecutionState.PENDING (queue position: 1)
2024-08-16 12:20:03,441 INFO dune_client.api.base waiting for query execution 01J5DZT5DN88FGSMNM892W0YRQ to complete: ExecutionState.PENDING (queue position: 1)
2024-08-16 12:20:04,586 INFO dune_client.api.base waiting for query execution 01J5DZT5DN88FGSMNM892W0YRQ to co

In [10]:
print(f"ch shape: {ch.shape}")
print(f"flip shape: {flip.shape}")
print(f"dune_df shape: {dune_df.shape}")

ch shape: (1502, 9)
flip shape: (174, 9)
dune_df shape: (2435, 9)


In [11]:
# Combine dfs
final_df = pd.concat([ch, flip, dune_df])

# Remove Dupes
final_df = final_df.drop_duplicates(subset=['chain_id','dt'], keep='first')

In [12]:
opstack_metadata = pd.read_csv('../op_chains_tracking/outputs/chain_metadata.csv')

opstack_metadata['chain_id'] = opstack_metadata['mainnet_chain_id']

meta_cols = ['is_op_chain','op_based_version', 'chain_id', 'alignment','chain_name', 'display_name']

# print("Columns in opstack_metadata:", opstack_metadata.columns)
# print("Columns in opstack_metadata[meta_cols]:", opstack_metadata[meta_cols].columns)
# print("Columns in final_df:", final_df.columns)


Columns in opstack_metadata: Index(['chain_name', 'display_name', 'mainnet_chain_id',
       'public_mainnet_launch_date', 'op_based_version', 'chain_type',
       'op_chain_start', 'has_mods', 'raas_deployer', 'rpc_url',
       'product_website', 'block_explorer_url', 'defillama_slug',
       'l2beat_slug', 'growthepie_origin_key', 'gas_token', 'chain_layer',
       'da_layer', 'output_root_layer', 'system_config_proxy',
       'oplabs_db_schema', 'goldsky_schema', 'dune_schema', 'flipside_schema',
       'oso_schema', 'batchinbox_from', 'batchinbox_to', 'outputoracle_from',
       'outputoracle_to_proxy', 'l1_standard_bridge', 'dispute_game_factory',
       'is_op_chain', 'alignment', 'chain_id'],
      dtype='object')
Columns in opstack_metadata[meta_cols]: Index(['is_op_chain', 'op_based_version', 'chain_id', 'alignment',
       'chain_name', 'display_name'],
      dtype='object')
Columns in final_df: Index(['dt', 'blockchain', 'name', 'layer', 'chain_id', 'num_raw_txs',
       'nu

In [13]:
final_enriched_df = final_df.merge(opstack_metadata[meta_cols], on='chain_id', how = 'left')
final_enriched_df['alignment'] = final_enriched_df['alignment'].fillna('Other EVMs')
final_enriched_df['is_op_chain'] = final_enriched_df['is_op_chain'].fillna(False)
final_enriched_df['display_name'] = final_enriched_df['display_name'].fillna(final_enriched_df['name'])

final_enriched_df = final_enriched_df.drop(columns=['name'])

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_15768/1364696242.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_enriched_df['is_op_chain'] = final_enriched_df['is_op_chain'].fillna(False)


In [14]:
final_enriched_df.sort_values(by=['dt','blockchain'], ascending =[False, False], inplace = True)

final_enriched_df.to_csv('outputs/'+query_name+'.csv', index=False)

In [15]:
final_enriched_df['chain_id'] = final_enriched_df['chain_id'].astype(int)
final_enriched_df['num_raw_txs'] = final_enriched_df['num_raw_txs'].astype(int)
final_enriched_df['num_success_txs'] = final_enriched_df['num_success_txs'].astype(int)
final_enriched_df['num_qualified_txs'] = final_enriched_df['num_qualified_txs'].fillna(0)
final_enriched_df['num_qualified_txs'] = final_enriched_df['num_qualified_txs'].astype(int)
# final_enriched_df.dtypes

In [16]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(final_enriched_df, query_name, unique_keys = ['chain_id','dt'])

Start Writing api_table_uploads.daily_evms_qualified_txs_counts_staging
Data loaded successfully to api_table_uploads.daily_evms_qualified_txs_counts_staging


2024-08-16 12:50:09,814 INFO google_bq_utils Append and upsert to api_table_uploads.daily_evms_qualified_txs_counts completed successfully
2024-08-16 12:50:09,980 INFO google_bq_utils Staging table oplabs-tools-data.api_table_uploads.daily_evms_qualified_txs_counts_staging deleted.
